In [1]:
# 导入库
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

In [2]:
# 导入包含特征、赔付概率和赔付金额的数据集（x, y_prob, y_amount）
data = pd.read_csv('C:/Users/taowu/Desktop/项目/保险赔付金额预测（nn-qrnn）/数据/my_data1.csv')  # 导入数据

# 创建LabelEncoder对象
label_encoder = LabelEncoder()

# 对指定列进行标签编码
data['veh_body_encoded'] = label_encoder.fit_transform(data['veh_body'])
data['area_encoded'] = label_encoder.fit_transform(data['area'])
data['amount_per_exposure'] = data['claimcst0'] / data['exposure']

# 输入特征和真实y值
x = data[['veh_value', 'veh_age', 'agecat', 'gender', 'veh_body_encoded', 'area_encoded']].values
y_prob = data[['clm']].values
y_amount = data[['amount_per_exposure']].values

# 划分训练集和测试集
x_train, x_test, y_prob_train, y_prob_test, y_amount_train, y_amount_test = train_test_split(x, y_prob, y_amount,
                                                                                             test_size=0.2,
                                                                                             random_state=42)

In [3]:
# 定义函数 t(p)
def t_function(p, T):
    return (T - p) / (1 - p)

# 定义 T
T = 0.95

In [10]:
# 将类别标签反转
y_prob_train_inverted = 1 - y_prob_train
# 使用逻辑回归模型
logistic_model = LogisticRegression()
# 训练逻辑回归模型
logistic_model.fit(x_train, y_prob_train_inverted.squeeze())
# 获取训练集逻辑回归模型的预测概率
predictions_A_train = logistic_model.predict_proba(x_train)[:, 1]
# 计算训练集的 tau
t_p_train = t_function(predictions_A_train, T)
# 计算测试集的 tau 
predictions_A_test = logistic_model.predict_proba(x_test)[:, 1]
t_p_test = t_function(predictions_A_test, T)
# 打印predictions_A、t_p、predictions_A_test、t_p_test
print("predictions_A:", predictions_A_train)
print("t_p_train:",t_p_train)
print("predictions_A_test:", predictions_A_test)
print("t_p_test:", t_p_test)

predictions_A: [0.90717235 0.93355933 0.92080237 ... 0.93550523 0.93560693 0.91975326]
t_p_train: [0.46136739 0.24744886 0.36866801 ... 0.22474336 0.22351898 0.37692175]
predictions_A_test: [0.92530429 0.93968977 0.93529386 ... 0.9365829  0.94805219 0.92018378]
t_p_test: [0.33061749 0.17095327 0.22727576 ... 0.21156915 0.03749555 0.37356089]
